In [ ]:
import pandas as pd
base_path = "C:\\400L Project\\analysis\\"
workbook = base_path + "WDIEXCEL.xlsx"
WDIframe = pd.read_excel(workbook, "Data")

print(WDIframe.head())

In [ ]:
print(WDIframe[WDIframe["Country Name"] == "Nigeria"])

In [ ]:
WDIframe_trimmed = WDIframe[["Country Name", "Indicator Name", "Indicator Code", "2017", "2018", "2019"]] # Trimmed WDI Dataset

In [ ]:
import openpyxl
WDIframe_trimmed.to_excel(base_path + 'WDI_EXCEL_TRIMMED.xlsx', sheet_name='Data')

In [ ]:
print(WDIframe_trimmed.head())

In [81]:
# Automate the insertion of the continents of each country
import pycountry_convert as pc
continents_for_each_row = [] # This holds the continent value for each row in the Trimmed dataset
trimmed_WDI_file_path = base_path + 'WDI_EXCEL_TRIMMED.xlsx'
trimmed_WDI_workframe = pd.read_excel(trimmed_WDI_file_path, "Data")

continents_code_names = {
'AF': 'Africa',
'AN': 'Antartica',
'AS': 'Asia',
'EU': 'Europe',
'NA': 'North America',
'OC': 'Oceania',
'SA': 'South America',
}

for index, row in trimmed_WDI_workframe.iterrows(): # Iterate through DataFrame rows
    try:
        country_code = pc.country_name_to_country_alpha2(row[1].strip(), cn_name_format="default") # Convert country name to ISO code
        continent_code = pc.country_alpha2_to_continent_code(country_code) # Convert country code to continent ISO code
        continent_name = continents_code_names[continent_code]
        
        continents_for_each_row.append(continent_name)
        
    except KeyError:
        continents_for_each_row.append('INVALID COUNTRY')
trimmed_WDI_workframe.insert(1, "Continent", continents_for_each_row, True)

   Unnamed: 0 Country Name                                     Indicator Name  \
0           0   Arab World  Access to clean fuels and technologies for coo...   
1           1   Arab World            Access to electricity (% of population)   
2           2   Arab World  Access to electricity, rural (% of rural popul...   
3           3   Arab World  Access to electricity, urban (% of urban popul...   
4           4   Arab World  Account ownership at a financial institution o...   

      Indicator Code       2017       2018  2019  
0     EG.CFT.ACCS.ZS        NaN        NaN   NaN  
1     EG.ELC.ACCS.ZS  90.283638  89.286856   NaN  
2  EG.ELC.ACCS.RU.ZS  81.102134  79.248100   NaN  
3  EG.ELC.ACCS.UR.ZS  97.467915  97.063959   NaN  
4     FX.OWN.TOTL.ZS  37.165211        NaN   NaN  


In [82]:
trimmed_WDI_workframe.to_excel(base_path + 'WDI_EXCEL_TRIMMED_WITH_CONTINENT.xlsx', sheet_name='Data', index=False)

In [80]:
print(trimmed_WDI_workframe.head())
#print(trimmed_WDI_workframe[["Country Name", "Continent"]])

   Unnamed: 0        Continent Country Name  \
0           0  INVALID COUNTRY   Arab World   
1           1  INVALID COUNTRY   Arab World   
2           2  INVALID COUNTRY   Arab World   
3           3  INVALID COUNTRY   Arab World   
4           4  INVALID COUNTRY   Arab World   

                                      Indicator Name     Indicator Code  \
0  Access to clean fuels and technologies for coo...     EG.CFT.ACCS.ZS   
1            Access to electricity (% of population)     EG.ELC.ACCS.ZS   
2  Access to electricity, rural (% of rural popul...  EG.ELC.ACCS.RU.ZS   
3  Access to electricity, urban (% of urban popul...  EG.ELC.ACCS.UR.ZS   
4  Account ownership at a financial institution o...     FX.OWN.TOTL.ZS   

        2017       2018  2019  
0        NaN        NaN   NaN  
1  90.283638  89.286856   NaN  
2  81.102134  79.248100   NaN  
3  97.467915  97.063959   NaN  
4  37.165211        NaN   NaN  


In [93]:
trimmed_WDI_with_continents = pd.read_excel(base_path + 'WDI_EXCEL_TRIMMED_WITH_CONTINENT.xlsx', "Data")
continent_select = trimmed_WDI_with_continents[["Country Name", "Continent"]] # Select continents and countries

africa = continent_select[continent_select["Continent"] == "Africa"]
south_ame = continent_select[continent_select["Continent"] == "South America"]
asia = continent_select[continent_select["Continent"] == "Asia"]

africa_trim = africa.drop_duplicates(subset='Country Name', keep = 'last')
south_ame_trim = south_ame.drop_duplicates(subset = 'Country Name', keep = 'last')
asia_trim = asia.drop_duplicates(subset = 'Country Name', keep = 'last')


continents_countries_only = pd.concat([africa_trim, south_ame_trim, asia_trim]) # Dataframe that has only countries in Africa, Asia, and South America
continents_countries_only = continents_countries_only.reset_index(drop=True, inplace=False)


<class 'pandas.core.frame.DataFrame'>


In [94]:
continents_countries_only.to_excel(base_path + 'AFRICA_ASIA_SOUTHAME_CONTRIES.xlsx', sheet_name='Data')

<class 'pandas.core.frame.DataFrame'>


In [139]:
# Script to merge two Excel files. The first file has the HDI of all countries. 
# I want to insert into the appropriate column in the selected countries dataframe, 
# the correspondng HDI value of that country that is found in the HDI_DATASET dataframe

hdi_dataset_frame = pd.read_excel(base_path + 'HDI_DATASET.xlsx', sheet_name='HDI_DATA')
# Read Excel file that has only countries in Africa, Asia, and South America
selected_continents_frame = pd.read_excel(base_path + 'AFRICA_ASIA_SOUTHAME_CONTRIES.xlsx', sheet_name='Data')

hdi_dataset_dict = hdi_dataset_frame.set_index('Country').T.to_dict('list') # Convert HDI dataframe to dictionary
selected_continents_frame['HDI'] = ''

for index, row in selected_continents_frame.iterrows():
    try:
        #print(selected_continents_frame.loc[ index, :], ' --', hdi_dataset_dict[row.values[1].strip()])
        selected_continents_frame.at[ index, 'HDI'] = hdi_dataset_dict[row.values[1].strip()][0]
        #selected_continents_frame.loc[3, 'HDI'] =
        #print(selected_continents_frame.loc[ index, :], ' -- ', hdi_dataset_dict[row[1].strip()])  
        
    except:
        selected_continents_frame.at[ index, 'HDI'] = -1


In [142]:
selected_continents_frame.to_excel(base_path + 'SELECTED_COUNTRIES_WITH_HDI.xlsx', sheet_name='Data', index=False)                                